## Experiment design

- Take a CAA-style Temporal Awareness explicit dataset : *data/raw/temporal_scope_caa.json*
- Create clean prompts from it following the template: *"{goal statement} {immediate}, and not {long-term}. Is this a short-term goal? The answer is "*
- Create corrupted prompts from it following the template *"{goal statement} {long-term}, and not {immediate}. Is this a short-term goal? The answer is"*
- Example:
    - Clean prompt: <code>"The goal is to get quick wins now, and not build sustainable advantage. Is this a short-term goal? The answer is""</code>
    - Clean answer: <code>"yes"</code>
    - Corrupted prompt: <code>"The goal is to build sustainable advantage, and not get quick wins now. Is this a short-term goal? The answer is"</code>
    - Corrupted answer: <code>"no"</code>
- *Note: Dataset is created from 6 examples and stored in data/raw/temporal_scope_for_attribution_patching.json*
- Run attribution patching as in TransformerLens demo: https://github.com/TransformerLensOrg/TransformerLens/blob/main/demos/Attribution_Patching_Demo.ipynb

In [ ]:
from pathlib import Path
import sys
import json

# from google.colab import drive
# drive.mount('/content/drive')
# !pwd # Returns /content
# ! cp "/content/drive/MyDrive/Colab Notebooks/attribution_patching.py" /content/attribution_patching.py
# ! cp "/content/drive/MyDrive/Colab Notebooks/temporal_scope_for_attribution_patching.json" /content/temporal_scope_for_attribution_patching.json
# import attribution_patching as attrib_patch

local_modules_path = str(Path('.').parent.resolve().parent.resolve() / "scripts" / "circuits")
if local_modules_path not in sys.path:
    sys.path.append(local_modules_path)
import attribution_patching as attrib_patch

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


### 1. Define the model of interest

In [2]:
MODEL_NAME = "phi-3"

### 2. Read the dataset, prepare inputs

In [ ]:
temporal_dataset = None
datasets_folder = Path('.').parent.resolve().parent.resolve() / "data" / "raw"
dataset_path = datasets_folder / "temporal_scope_for_attribution_patching.json"
with open(dataset_path) as f:
    temporal_dataset = json.load(f)

clean_prompts = []
clean_answers = []
corrupted_prompts = []
corrupted_answers = []

for sample in temporal_dataset:
    clean_question = question=sample["clean"]["question"]
    clean_prompts.append(clean_question)
    corrupted_question = question=sample["corrupted"]["question"]
    corrupted_prompts.append(corrupted_question)
    clean_answers.append(sample["clean"]["answer"])
    corrupted_answers.append(sample["corrupted"]["answer"])

### Run Attribution patching for residual stream

In [4]:
atp_algorithm = attrib_patch.AttributionPatching(MODEL_NAME,
                                                 clean_prompts, clean_answers,
                                                 corrupted_prompts, corrupted_answers)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model phi-3 into HookedTransformer
Clean string 0 <s> The goal is to prioritize addressing today's urgent needs, and not investing in tomorrow's opportunities. Is this a short-term goal? The answer is
Corrupted string 0 <s> The goal is to prioritize investing in tomorrow's opportunities, and not addressing today's urgent needs. Is this a short-term goal? The answer is
Clean answers tensor([4874, 4874, 4874, 4874, 4874, 4874])
Corrupted answers tensor([694, 694, 694, 694, 694, 694])


In [5]:
residual_attr, residual_labels = atp_algorithm.patch_residual()

Clean logit TOP-3: [': True yes', 'yes True:', 'yes Yes no', 'yes Yes no', 'True Yes yes', 'yes no Yes']

Corrupted logit TOP-3: [": no '", 'no False No', 'no yes False', 'no yes a', 'no False:', 'no yes False']
Clean logit diff: 3.0006
Corrupted logit diff: -2.2510


In [6]:
fig = px.imshow(residual_attr.cpu(), y=residual_labels,
                color_continuous_scale="RdBu",
                color_continuous_midpoint=0.0,
                aspect="auto",
                title="Residual Attribution Patching")
fig.layout.xaxis.title.text = "Position"
fig.layout.yaxis.title.text = "Component"

fig.show()

### Run Attribution patching for layer output stream

In [7]:
layer_out_attr, layer_out_labels = atp_algorithm.patch_layer_out()

Clean logit TOP-3: [': True yes', 'yes True:', 'yes Yes no', 'yes Yes no', 'True Yes yes', 'yes no Yes']

Corrupted logit TOP-3: [": no '", 'no False No', 'no yes False', 'no yes a', 'no False:', 'no yes False']
Clean logit diff: 3.0006
Corrupted logit diff: -2.2510


In [8]:
fig = px.imshow(residual_attr.cpu(), y=residual_labels,
                color_continuous_scale="RdBu",
                color_continuous_midpoint=0.0,
                aspect="auto",
                title="Layer Output Attribution Patching")
fig.layout.xaxis.title.text = "Position"
fig.layout.yaxis.title.text = "Component"

fig.show()